In [212]:
import pandas as pd


In [213]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

import sqlite3

In [214]:
data1=pd.read_csv("clear_hawaii_measurements.csv")
data1.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [215]:
data2=pd.read_csv("resources/hawaii_stations.csv")
data2.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [216]:
engine1 = create_engine("sqlite:///measurements.sqlite")
engine2 = create_engine("sqlite:///stations.sqlite")

In [217]:
conn1 = engine1.connect()
conn2 = engine2.connect()
conn1.execute("DROP TABLE IF EXISTS weather")
conn2.execute("DROP TABLE IF EXISTS stations")

In [218]:
Base1 = declarative_base()
Base2 = declarative_base()

In [219]:
class Measurements(Base1):
    __tablename__ = 'weather'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp= Column(Numeric)
    tobs=Column(Integer)

In [220]:
class Stations(Base2):
    __tablename__ = 'stations'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Numeric)
    longitude= Column(Numeric)
    elevation=Column(Numeric)

In [221]:
Base1.metadata.create_all(engine1)

In [222]:
Base2.metadata.create_all(engine2)

In [223]:
measurements_data = data1.to_dict(orient='records')

In [224]:
stations_data = data2.to_dict(orient='records')

In [225]:
print(stations_data[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [226]:
print(measurements_data[:5])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]


In [227]:
metadata1 = MetaData(bind=engine1)
metadata1.reflect()

metadata2 = MetaData(bind=engine2)
metadata2.reflect()

In [228]:
table1 = sqlalchemy.Table('weather', metadata1, autoload=True)
table2 = sqlalchemy.Table('stations', metadata2, autoload=True)
table1

Table('weather', MetaData(bind=Engine(sqlite:///measurements.sqlite)), Column('id', INTEGER(), table=<weather>, primary_key=True, nullable=False), Column('station', TEXT(), table=<weather>), Column('date', TEXT(), table=<weather>), Column('prcp', NUMERIC(), table=<weather>), Column('tobs', INTEGER(), table=<weather>), schema=None)

In [229]:
conn1.execute(table1.delete())
conn2.execute(table2.delete())

In [232]:
conn1.execute(table1.insert(), measurements_data)

In [233]:
conn2.execute(table2.insert(), stations_data)

In [235]:
conn1.execute("select * from weather limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0, 63),
 (3, 'USC00519397', '2010-01-03', 0, 74),
 (4, 'USC00519397', '2010-01-04', 0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]